## **Mount Google Drive**

In [29]:
from google.colab import drive
drive.mount("/content/drive")

import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Path Input/Output**

In [30]:
ROOT = "/content/drive/MyDrive/bigdata_final_project"
RAW_OUT = f"{ROOT}/raw"
LOG_OUT = f"{ROOT}/etl_pipeline/extract.log"

os.makedirs(RAW_OUT, exist_ok=True)
os.makedirs(os.path.dirname(LOG_OUT), exist_ok=True)

SRC1 = "/content/drive/MyDrive/bigdata_final_project/datalake/Salinan dfTransjakarta180kRows.csv"
SRC2 = "/content/drive/MyDrive/bigdata_final_project/datalake/Salinan weather_hourly_raw_2023_04_01_05_01.csv"

# **EXTRACT**

In [31]:
# IMPORT
import os, json, time, pandas as pd
from datetime import datetime

In [32]:
# FUNGSI EXTRACT SOURCE 1
def extract_etl_source1():
    t0 = time.time()
    start = datetime.now().isoformat()
    df = pd.read_csv(SRC1)  # baca mentah
    r, c = df.shape
    out = f"{RAW_OUT}/transjakarta_raw.csv"
    df.to_csv(out, index=False)  # simpan raw
    log = {
        "source":"TransJakarta",
        "rows":r,
        "cols":c,
        "size":os.path.getsize(out),
        "time":round(time.time()-t0,2),
        "start":start
        }
    with open(LOG_OUT,"a") as f: f.write(json.dumps(log)+"\n")
    return log

In [33]:
# FUNGSI EXTRACT SOURCE 2
def extract_etl_source2():
    t0 = time.time()
    start = datetime.now().isoformat()
    df = pd.read_csv(SRC2)  # baca mentah
    r, c = df.shape
    out = f"{RAW_OUT}/weather_raw.csv"
    df.to_csv(out, index=False)  # simpan raw
    log = {
        "source":"Meteostat (API)",
        "lokasi":"Jakarta",
        "periode":"2023-04-01 s.d. 2023-05-01",
        "granularitas":"Hourly",
        "rows":r,
        "cols":c,
        "size":os.path.getsize(out),
        "time":round(time.time()-t0,2),
        "start":start
        }
    with open(LOG_OUT,"a") as f: f.write(json.dumps(log)+"\n")
    return log

In [34]:
# RUN EXTRACT
log1 = extract_etl_source1()
print("============== LOG ETL ==============")
print("Source:", log1["source"])
print("Rows:", log1["rows"])
print("Cols:", log1["cols"])
print("Size:", log1["size"])
print("Start:", log1["start"])
print("")

log2 = extract_etl_source2()
print("============== LOG ETL ==============")
print("Source:", log2["source"])
print("Lokasi:", log2["lokasi"])
print("Periode:", log2["periode"])
print("Granularitas:", log2["granularitas"])
print("Rows:", log2["rows"])
print("Cols:", log2["cols"])
print("Size:", log2["size"])
print("Start:", log2["start"])
print("")

print("============== Raw files ==============")
for f in os.listdir(RAW_OUT):
    if f.endswith(".csv"):
        print(f)

============== LOG ETL ==============
Source: TransJakarta
Rows: 189500
Cols: 22
Size: 44825012
Start: 2026-01-06T16:13:55.646415

============== LOG ETL ==============
Source: Meteostat (API)
Lokasi: Jakarta
Periode: 2023-04-01 s.d. 2023-05-01
Granularitas: Hourly
Rows: 744
Cols: 12
Size: 46882
Start: 2026-01-06T16:14:01.587650

============== Raw files ==============
dfTransjakarta180kRows.csv
weather_hourly_raw_2023_04_01_05_01.csv
transjakarta_raw.csv
weather_raw.csv


# **TRANSFORM**

**Import Library**

In [35]:
import os
import re
import time
import json
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

## **Data Cleaning**

**Helper (IQR Outlier)**

In [36]:
# HELPER (IQR OUTLIER)
def iqr_clip(series: pd.Series, k: float = 1.5):
    s = pd.to_numeric(series, errors="coerce")
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    if pd.isna(iqr) or iqr == 0:
        return s, 0, None, None
    lo = q1 - k * iqr
    hi = q3 + k * iqr
    before = s.copy()
    clipped = s.clip(lower=lo, upper=hi)
    out_cnt = int(((before < lo) | (before > hi)).sum(skipna=True))
    return clipped, out_cnt, float(lo), float(hi)

**Cleaning Dataset TransJakarta**

In [37]:
def clean_transjakarta(df: pd.DataFrame):
    report = {"dataset": "TransJakarta"}

    before_rows, before_cols = df.shape

    # DROP DUPLICATES (PK: transID)
    if "transID" in df.columns:
        dup_cnt = int(df.duplicated(subset=["transID"]).sum())
        df = df.drop_duplicates(subset=["transID"], keep="first")
        report["dup_removed_by_transID"] = dup_cnt
    else:
        report["dup_removed_by_transID"] = None

    # STANDARDIZE DATETIME + FILL NaT DENGAN MODE
    dt_cols = ["tapInTime", "tapOutTime", "payCardBirthDate"]
    for col in dt_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            mode_val = df[col].mode(dropna=True)
            if not mode_val.empty:
                df[col] = df[col].fillna(mode_val.iloc[0])

    # MISSING VALUE
    # numerik -> median (fallback 0 jika median NaN)
    num_cols = [
        "tapInStopsLat","tapInStopsLon","tapOutStopsLat","tapOutStopsLon",
        "stopStartSeq","stopEndSeq","payAmount","corridorID","tapInStops","tapOutStops"
    ]
    for col in num_cols:
        if col in df.columns:
            s = pd.to_numeric(df[col], errors="coerce")
            med = s.median(skipna=True)
            if pd.isna(med):
                med = 0
            df[col] = s.fillna(med)

    # kategorikal -> "unknown"
    cat_cols = ["payCardBank","payCardName","payCardSex","corridorName","direction","tapInStopsName","tapOutStopsName"]
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype("string").fillna("unknown")

    # id -> string, missing -> "unknown"
    for col in ["transID", "payCardID"]:
        if col in df.columns:
            df[col] = df[col].astype("string").fillna("unknown")

    # OUTLIER (IQR CLIP)
    outlier_cols = ["payAmount","tapInStopsLat","tapInStopsLon","tapOutStopsLat","tapOutStopsLon"]
    out_info = {}
    for col in outlier_cols:
        if col in df.columns:
            clipped, out_cnt, lo, hi = iqr_clip(df[col], k=1.5)
            df[col] = clipped
            out_info[col] = {"outliers": out_cnt, "lo": lo, "hi": hi}
    report["iqr_clip"] = out_info

    after_rows, _ = df.shape
    report["rows_before"] = int(before_rows)
    report["rows_after"] = int(after_rows)
    report["cols"] = int(before_cols)

    return df, report

**Cleaning Dataset Weather**

In [38]:
def clean_weather(df: pd.DataFrame):
    report = {"dataset": "Weather"}

    before_rows, before_cols = df.shape

    # DROP KOLOM KOSONG
    drop_cols = ["snow", "wpgt", "tsun"]
    exist_drop = [c for c in drop_cols if c in df.columns]
    dropped = []
    for c in exist_drop:
        if df[c].isna().all():
            dropped.append(c)
    if dropped:
        df = df.drop(columns=dropped)
    report["dropped_all_null_cols"] = dropped

    # DROP DUPLICATES (PK: time)
    if "time" in df.columns:
        dup_cnt = int(df.duplicated(subset=["time"]).sum())
        df = df.drop_duplicates(subset=["time"], keep="first")
        report["dup_removed_by_time"] = dup_cnt
    else:
        report["dup_removed_by_time"] = None

    # STANDARDIZE DATETIME FORMAT
    if "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"], errors="coerce")

    # MISSING VALUE (numerik -> median) (tanpa kolom yang sudah di-drop)
    num_cols = ["temp","dwpt","rhum","prcp","wdir","wspd","pres","coco"]
    for col in num_cols:
        if col in df.columns:
            s = pd.to_numeric(df[col], errors="coerce")
            med = s.median(skipna=True)
            df[col] = s.fillna(med)

    # OUTLIER (IQR CLIP)
    outlier_cols = ["temp","rhum","prcp","wspd","pres"]
    out_info = {}
    for col in outlier_cols:
        if col in df.columns:
            clipped, out_cnt, lo, hi = iqr_clip(df[col], k=1.5)
            df[col] = clipped
            out_info[col] = {"outliers": out_cnt, "lo": lo, "hi": hi}
    report["iqr_clip"] = out_info

    after_rows, _ = df.shape
    report["rows_before"] = int(before_rows)
    report["rows_after"] = int(after_rows)
    report["cols_before"] = int(before_cols)
    report["cols_after"] = int(df.shape[1])

    return df, report

**Output Hasil Cleaning**

In [39]:
t0 = time.time()

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

df_tj_raw = pd.read_csv(f"{RAW_OUT}/transjakarta_raw.csv")
df_w_raw  = pd.read_csv(f"{RAW_OUT}/weather_raw.csv")

df_tj_clean, rep_tj = clean_transjakarta(df_tj_raw)
df_w_clean, rep_w   = clean_weather(df_w_raw)

print("======== LOG ETL CLEANING ========")
print("TransJakarta:", rep_tj["rows_before"], "->", rep_tj["rows_after"])
print("TransJakarta dup removed:", rep_tj["dup_removed_by_transID"])
print("TransJakarta cols:", rep_tj["cols"])
print("Weather:", rep_w["rows_before"], "->", rep_w["rows_after"])
print("Weather cols:", rep_w["cols_after"])
print("Weather dup removed:", rep_w["dup_removed_by_time"])
print("Exec seconds:", round(time.time() - t0, 2))
print("")

print("Tabel TransJakarta Clean:")
display(df_tj_clean.head(5))

print("Tabel Weather Clean:")
display(df_w_clean.head(5))


======== LOG ETL CLEANING ========
TransJakarta: 189500 -> 189500
TransJakarta dup removed: 0
TransJakarta cols: 22
Weather: 744 -> 744
Weather cols: 9
Weather dup removed: 0
Exec seconds: 2.29

Tabel TransJakarta Clean:


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount
0,VRPJ892P3M98RA,3561407960318444,dki,"Dr. Janet Nashiruddin, M.Ak",M,1970-01-01 00:00:00.000002010,4.0,Pulo Gadung 2 - Tosari,1.0,0.0,Pemuda Rawamangun,-6.193488,106.89165,12,2023-04-03 06:53:02,0.0,Layur,-6.193539,106.89909,13.0,2023-04-03 07:13:28,3500.0
1,ZWCH834I6M26HS,347728053419394,emoney,Balamantri Rahayu,M,1970-01-01 00:00:00.000002002,8.0,Kp. Rambutan - Taman Wiladatika,1.0,0.0,Sekolah Islam PB Soedirman,-6.314293,106.86298,27,2023-04-03 05:59:19,0.0,Jln. Ar Ridho,-6.308148,106.86935,30.0,2023-04-03 06:57:06,0.0
2,YRLD835V6L82GO,377105453850671,emoney,Dian Mustofa,F,1970-01-01 00:00:00.000001993,8.0,Bekasi Barat - Blok M,1.0,0.0,Mall Metropolitan,-6.247861,106.99215,6,2023-04-03 05:13:24,0.0,Bandar Djakarta Bekasi,-6.227085,106.99683,9.0,2023-04-03 06:01:23,8750.0
3,ZZBX143N6N83HQ,4486493302356581,dki,"Cut Janet Suryatmi, M.Ak",M,1970-01-01 00:00:00.000001980,8.0,Batusari - Grogol,1.0,0.0,Sbr. Jln. Angsana Kemanggisan,-6.194813,106.78213,16,2023-04-03 05:20:24,0.0,Yayasan Alkahfi Jakbar,-6.198896,106.76889,26.0,2023-04-03 06:01:25,3500.0
4,EWEG491A2W45DR,30139379978125,bni,"dr. Mulyanto Pudjiastuti, M.Ak",F,1970-01-01 00:00:00.000001997,8.0,unknown,0.0,0.0,BRI Menteng,-6.186792,106.83514,2,2023-04-03 06:00:54,0.0,Perpustakaan Nasional,-6.180673,106.82643,5.0,2023-04-03 06:47:32,3500.0


Tabel Weather Clean:


,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco
0,2023-04-01 00:00:00,26.8,22.6,78.0,0.0,250.0,5.4,1010.8,3.0
1,2023-04-01 01:00:00,26.1,22.8,82.0,0.0,255.0,18.4,1010.7,3.0
2,2023-04-01 02:00:00,27.0,23.2,80.0,0.0,254.0,18.0,1010.9,3.0
3,2023-04-01 03:00:00,26.2,23.5,85.0,0.0,270.0,9.4,1011.9,3.0
4,2023-04-01 04:00:00,29.0,24.1,75.0,0.0,267.0,18.0,1010.2,3.0


## **Standarisasi Data**

In [40]:
# df_tj_clean, df_w_clean
print(df_tj_clean.shape, df_w_clean.shape)

(189500, 22) (744, 9)


**Helper - SNAKE_CASE**

In [41]:
def to_snake(name: str) -> str:
    name = name.strip()
    name = re.sub(r"[^0-9a-zA-Z]+", "_", name)
    name = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", name)
    name = re.sub(r"_+", "_", name)
    return name.lower().strip("_")

**Rename Kolom**

In [42]:
df_tj_std = df_tj_clean.copy()
df_w_std  = df_w_clean.copy()

df_tj_std.columns = [to_snake(c) for c in df_tj_std.columns]
df_w_std.columns  = [to_snake(c) for c in df_w_std.columns]

print("TJ (TransJakarta) cols sample:", df_tj_std.columns[:10].tolist())
print("W (Weather) cols sample:", df_w_std.columns.tolist())

TJ (TransJakarta) cols sample: ['trans_id', 'pay_card_id', 'pay_card_bank', 'pay_card_name', 'pay_card_sex', 'pay_card_birth_date', 'corridor_id', 'corridor_name', 'direction', 'tap_in_stops']
W (Weather) cols sample: ['time', 'temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'coco']


**Konsistensi Tipe Data (TransJakarta)**

In [43]:
def ensure_types_tj(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # id
    for c in ["trans_id", "pay_card_id"]:
        if c in df.columns:
            df[c] = df[c].astype("string")

    # datetime
    for c in ["tap_in_time", "tap_out_time", "pay_card_birth_date"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")

    # integer-like
    int_cols = ["corridor_id", "tap_in_stops", "tap_out_stops", "stop_start_seq", "stop_end_seq"]
    for c in int_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype("Int64")

    # float-like
    float_cols = ["pay_amount", "tap_in_stops_lat", "tap_in_stops_lon", "tap_out_stops_lat", "tap_out_stops_lon"]
    for c in float_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype(float)

    # categorical
    cat_cols = ["pay_card_bank","pay_card_name","pay_card_sex","corridor_name","direction","tap_in_stops_name","tap_out_stops_name"]
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].astype("string")

    return df

df_tj_std = ensure_types_tj(df_tj_std)
df_tj_std.dtypes.head(15)

,0
trans_id,string[python]
pay_card_id,string[python]
pay_card_bank,string[python]
pay_card_name,string[python]
pay_card_sex,string[python]
pay_card_birth_date,datetime64[ns]
corridor_id,Int64
corridor_name,string[python]
direction,string[python]
tap_in_stops,Int64


**Konsistensi Tipe Data (Weather)**

In [44]:
def ensure_types_w(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"], errors="coerce")

    num_cols = ["temp","dwpt","rhum","prcp","wdir","wspd","pres","coco"]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype(float)

    return df

df_w_std = ensure_types_w(df_w_std)
df_w_std.dtypes

,0
time,datetime64[ns]
temp,float64
dwpt,float64
rhum,float64
prcp,float64
wdir,float64
wspd,float64
pres,float64
coco,float64


**Normalisasi**

In [45]:
def add_minmax_norm(df: pd.DataFrame, cols: list, suffix: str = "_norm") -> pd.DataFrame:
    df = df.copy()
    use = [c for c in cols if c in df.columns]
    if len(use) == 0:
        return df

    scaler = MinMaxScaler()
    norm = scaler.fit_transform(df[use])
    for i, c in enumerate(use):
        df[f"{c}{suffix}"] = norm[:, i]
    return df

# TransJakarta: pay_amount, tap_in_stops_lat
df_tj_std = add_minmax_norm(df_tj_std, ["pay_amount", "tap_in_stops_lat"])

# Weather: temp, rhum
df_w_std = add_minmax_norm(df_w_std, ["temp", "rhum"])

print("TJ norm cols:", [c for c in df_tj_std.columns if c.endswith("_norm")])
print("W norm cols:", [c for c in df_w_std.columns if c.endswith("_norm")])

TJ norm cols: ['pay_amount_norm', 'tap_in_stops_lat_norm']
W norm cols: ['temp_norm', 'rhum_norm']


**Encoding Kategorical**

In [46]:
def label_encode_with_map(df: pd.DataFrame, col: str):
    s = df[col].astype("string").fillna("unknown")
    cats = pd.Index(s.unique()).sort_values()
    mapping = {v: int(i) for i, v in enumerate(cats)}
    encoded = s.map(mapping).astype("Int64")
    return encoded, mapping

def encode_cols(df: pd.DataFrame, cols: list):
    df = df.copy()
    maps = {}
    for c in cols:
        if c in df.columns:
            df[f"{c}_enc"], m = label_encode_with_map(df, c)
            maps[c] = m
    return df, maps

tj_cat = ["pay_card_bank", "pay_card_sex", "corridor_name", "direction"]
df_tj_std, tj_maps = encode_cols(df_tj_std, tj_cat)

w_cat = ["coco"]
df_w_std, w_maps = encode_cols(df_w_std, w_cat)

print("TJ encoded:", [c for c in df_tj_std.columns if c.endswith("_enc")])
print("W encoded:", [c for c in df_w_std.columns if c.endswith("_enc")])

TJ encoded: ['pay_card_bank_enc', 'pay_card_sex_enc', 'corridor_name_enc', 'direction_enc']
W encoded: ['coco_enc']


**Output Standarisasi**

In [47]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
summary = pd.DataFrame([
    {"dataset":"TransJakarta","rows":df_tj_std.shape[0],"cols":df_tj_std.shape[1]},
    {"dataset":"Weather","rows":df_w_std.shape[0],"cols":df_w_std.shape[1]},
])
display(summary)

print("Preview TransJakarta Standardized:")
display(df_tj_std.head(5))

print("Preview Weather Standardized:")
display(df_w_std.head(5))

,dataset,rows,cols
0,TransJakarta,189500,28
1,Weather,744,12


Preview TransJakarta Standardized:


,trans_id,pay_card_id,pay_card_bank,pay_card_name,pay_card_sex,pay_card_birth_date,corridor_id,corridor_name,direction,tap_in_stops,tap_in_stops_name,tap_in_stops_lat,tap_in_stops_lon,stop_start_seq,tap_in_time,tap_out_stops,tap_out_stops_name,tap_out_stops_lat,tap_out_stops_lon,stop_end_seq,tap_out_time,pay_amount,pay_amount_norm,tap_in_stops_lat_norm,pay_card_bank_enc,pay_card_sex_enc,corridor_name_enc,direction_enc
0,VRPJ892P3M98RA,3561407960318444,dki,"Dr. Janet Nashiruddin, M.Ak",M,1970-01-01 00:00:00.000002010,4,Pulo Gadung 2 - Tosari,1.0,0,Pemuda Rawamangun,-6.193488,106.89165,12,2023-04-03 06:53:02,0,Layur,-6.193539,106.89909,13,2023-04-03 07:13:28,3500.0,0.4,0.607159,2,1,120,1
1,ZWCH834I6M26HS,347728053419394,emoney,Balamantri Rahayu,M,1970-01-01 00:00:00.000002002,8,Kp. Rambutan - Taman Wiladatika,1.0,0,Sekolah Islam PB Soedirman,-6.314293,106.86298,27,2023-04-03 05:59:19,0,Jln. Ar Ridho,-6.308148,106.86935,30,2023-04-03 06:57:06,0.0,0.0,0.151099,3,1,80,1
2,YRLD835V6L82GO,377105453850671,emoney,Dian Mustofa,F,1970-01-01 00:00:00.000001993,8,Bekasi Barat - Blok M,1.0,0,Mall Metropolitan,-6.247861,106.99215,6,2023-04-03 05:13:24,0,Bandar Djakarta Bekasi,-6.227085,106.99683,9,2023-04-03 06:01:23,8750.0,1.0,0.401892,3,0,5,1
3,ZZBX143N6N83HQ,4486493302356581,dki,"Cut Janet Suryatmi, M.Ak",M,1970-01-01 00:00:00.000001980,8,Batusari - Grogol,1.0,0,Sbr. Jln. Angsana Kemanggisan,-6.194813,106.78213,16,2023-04-03 05:20:24,0,Yayasan Alkahfi Jakbar,-6.198896,106.76889,26,2023-04-03 06:01:25,3500.0,0.4,0.602157,2,1,4,1
4,EWEG491A2W45DR,30139379978125,bni,"dr. Mulyanto Pudjiastuti, M.Ak",F,1970-01-01 00:00:00.000001997,8,unknown,0.0,0,BRI Menteng,-6.186792,106.83514,2,2023-04-03 06:00:54,0,Perpustakaan Nasional,-6.180673,106.82643,5,2023-04-03 06:47:32,3500.0,0.4,0.632438,0,0,216,0


Preview Weather Standardized:


,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,temp_norm,rhum_norm,coco_enc
0,2023-04-01 00:00:00,26.8,22.6,78.0,0.0,250.0,5.4,1010.8,3.0,0.395349,0.628319,4
1,2023-04-01 01:00:00,26.1,22.8,82.0,0.0,255.0,18.4,1010.7,3.0,0.341085,0.699115,4
2,2023-04-01 02:00:00,27.0,23.2,80.0,0.0,254.0,18.0,1010.9,3.0,0.410853,0.663717,4
3,2023-04-01 03:00:00,26.2,23.5,85.0,0.0,270.0,9.4,1011.9,3.0,0.348837,0.752212,4
4,2023-04-01 04:00:00,29.0,24.1,75.0,0.0,267.0,18.0,1010.2,3.0,0.565891,0.575221,4


## **Data Enrichment dan Feature Engineering**

In [48]:
# (df_tj_std dan df_w_std)
need_tj = ["tap_in_time", "tap_out_time", "pay_amount", "tap_in_stops_lat", "tap_in_stops_lon", "tap_out_stops_lat", "tap_out_stops_lon"]
need_w  = ["time", "temp", "rhum", "prcp", "wspd", "pres", "coco"]

print("Missing TJ:", [c for c in need_tj if c not in df_tj_std.columns])
print("Missing W :", [c for c in need_w  if c not in df_w_std.columns])

Missing TJ: []
Missing W : []


**Siapkan datetime dan sort untuk merge_asof**

In [49]:
df_tj_std["tap_in_time"]  = pd.to_datetime(df_tj_std["tap_in_time"], errors="coerce")
df_tj_std["tap_out_time"] = pd.to_datetime(df_tj_std["tap_out_time"], errors="coerce")

df_w_std["time"] = pd.to_datetime(df_w_std["time"], errors="coerce")

df_tj_std = df_tj_std.sort_values("tap_in_time").reset_index(drop=True)
df_w_std  = df_w_std.sort_values("time").reset_index(drop=True)

print("TJ time range:", df_tj_std["tap_in_time"].min(), "->", df_tj_std["tap_in_time"].max())
print("W  time range:", df_w_std["time"].min(), "->", df_w_std["time"].max())

TJ time range: 2023-04-01 05:14:35 -> 2023-04-30 21:58:49
W  time range: 2023-04-01 00:00:00 -> 2023-05-01 23:00:00


**Enrichment: Merge Transaksi san Weather Berdasarkan Waktu**

In [50]:
# join: ambil weather terdekat terhadap tap_in_time (toleransi 1 jam)
df_enriched = pd.merge_asof(
    df_tj_std,
    df_w_std,
    left_on="tap_in_time",
    right_on="time",
    direction="nearest",
    tolerance=pd.Timedelta("1H")
)

print("ENRICHED SHAPE:", df_enriched.shape)
print("Weather matched (temp not null):", int(df_enriched["temp"].notna().sum()))

/tmp/ipython-input-401087497.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  tolerance=pd.Timedelta("1H")


ENRICHED SHAPE: (189500, 40)
Weather matched (temp not null): 189500


**Future Engineering: 5 Fitur**

In [51]:
def haversine_km(lat1, lon1, lat2, lon2):
    lat1 = np.radians(pd.to_numeric(lat1, errors="coerce"))
    lon1 = np.radians(pd.to_numeric(lon1, errors="coerce"))
    lat2 = np.radians(pd.to_numeric(lat2, errors="coerce"))
    lon2 = np.radians(pd.to_numeric(lon2, errors="coerce"))

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371.0 * c  # km

df = df_enriched.copy()

# Fitur 1: hour
df["feat_hour"] = df["tap_in_time"].dt.hour

# Fitur 2: day_of_week (0=Mon)
df["feat_dow"] = df["tap_in_time"].dt.dayofweek

# Fitur 3: is_weekend
df["feat_is_weekend"] = df["feat_dow"].isin([5, 6]).astype("Int64")

# Fitur 4: durasi perjalanan (menit)
df["feat_trip_minutes"] = (df["tap_out_time"] - df["tap_in_time"]).dt.total_seconds() / 60.0

# Fitur 5: jarak (km) dari koordinat tap in ke tap out
df["feat_dist_km"] = haversine_km(
    df["tap_in_stops_lat"], df["tap_in_stops_lon"],
    df["tap_out_stops_lat"], df["tap_out_stops_lon"]
)

# Fitur 6: tarif per km
df["feat_fare_per_km"] = pd.to_numeric(df["pay_amount"], errors="coerce") / df["feat_dist_km"].replace(0, np.nan)

# Fitur 7: flag hujan (prcp > 0)
df["feat_is_rain"] = (pd.to_numeric(df["prcp"], errors="coerce").fillna(0) > 0).astype("Int64")

# Fitur 8: index panas sederhana (temp + rhum)
df["feat_temp_rhum_index"] = pd.to_numeric(df["temp"], errors="coerce") + (pd.to_numeric(df["rhum"], errors="coerce") / 100.0)

# rapikan nilai tak valid
df["feat_trip_minutes"] = df["feat_trip_minutes"].clip(lower=0)
df.loc[df["feat_trip_minutes"].isna(), "feat_trip_minutes"] = df["feat_trip_minutes"].median(skipna=True)

print("FE DONE. TOTAL COLS:", df.shape[1])

FE DONE. TOTAL COLS: 48


**Output Data Enrichment dan Feature Engineering**

In [52]:
print("ROWS:", df.shape[0])
print("COLS:", df.shape[1])

feat_cols = [
    "feat_hour","feat_dow","feat_is_weekend","feat_trip_minutes",
    "feat_dist_km","feat_fare_per_km","feat_is_rain","feat_temp_rhum_index"
]

print("Feature columns available:", [c for c in feat_cols if c in df.columns])
display(df[["tap_in_time","tap_out_time","pay_amount","temp","rhum","prcp"] + feat_cols].head(10))

ROWS: 189500
COLS: 48
Feature columns available: ['feat_hour', 'feat_dow', 'feat_is_weekend', 'feat_trip_minutes', 'feat_dist_km', 'feat_fare_per_km', 'feat_is_rain', 'feat_temp_rhum_index']


,tap_in_time,tap_out_time,pay_amount,temp,rhum,prcp,feat_hour,feat_dow,feat_is_weekend,feat_trip_minutes,feat_dist_km,feat_fare_per_km,feat_is_rain,feat_temp_rhum_index
0,2023-04-01 05:14:35,2023-04-01 07:52:46,3500.0,28.2,77.0,0.3,5,5,1,158.183333,4.070741,859.794244,1,28.97
1,2023-04-01 05:17:20,2023-04-01 06:59:11,0.0,28.2,77.0,0.3,5,5,1,101.850000,3.508057,0.000000,1,28.97
2,2023-04-01 05:30:41,2023-04-01 06:04:48,3500.0,29.6,70.0,0.2,5,5,1,34.116667,3.353553,1043.669216,1,30.30
3,2023-04-01 05:50:47,2023-04-01 08:16:57,3500.0,29.6,70.0,0.2,5,5,1,146.166667,6.463954,541.464242,1,30.30
4,2023-04-01 06:27:54,2023-04-01 06:50:11,0.0,29.6,70.0,0.2,6,5,1,22.283333,1.972499,0.000000,1,30.30
5,2023-04-01 06:35:14,2023-04-01 09:27:40,8750.0,27.9,76.0,2.0,6,5,1,172.433333,2.021063,4329.405554,1,28.66
6,2023-04-01 06:46:38,2023-04-01 07:44:44,3500.0,27.9,76.0,2.0,6,5,1,58.100000,2.414858,1449.360599,1,28.66
7,2023-04-01 07:49:59,2023-04-01 09:04:35,3500.0,27.7,80.0,0.5,7,5,1,74.600000,6.490575,539.243429,1,28.50
8,2023-04-01 08:37:05,2023-04-01 10:51:17,0.0,29.8,62.0,0.0,8,5,1,134.200000,6.320959,0.000000,0,30.42
9,2023-04-01 09:00:14,2023-04-01 11:11:00,0.0,29.8,62.0,0.0,9,5,1,130.766667,0.975394,0.000000,0,30.42


## **Validasi Kualitas Data**

In [54]:
dfv = df.copy()

results = []

def add_result(rule, passed, detail):
    results.append({"rule": rule, "passed": int(bool(passed)), "detail": detail})

# 1. UNIQUENESS
passed = True
if "trans_id" in dfv.columns:
    dup = dfv["trans_id"].duplicated().sum()
    passed = dup == 0
    add_result("uniqueness_trans_id", passed, f"duplicates={dup}")
else:
    add_result("uniqueness_trans_id", False, "column trans_id not found")

# 2. NULL CHECK
passed = True
critical = ["trans_id","tap_in_time","tap_out_time","pay_amount","temp","rhum","prcp"]
cols = [c for c in critical if c in dfv.columns]
if cols:
    nulls = dfv[cols].isna().sum().sum()
    passed = nulls == 0
    add_result("null_check_critical", passed, f"null_total={nulls} cols={cols}")
else:
    add_result("null_check_critical", False, "no critical columns found")

# 3. RANGE CHECK
passed = True
checks = []
if "pay_amount" in dfv.columns:
    f = ((dfv["pay_amount"] < 0) | (dfv["pay_amount"] > 50000)).sum()
    checks.append(("pay_amount_range", f))
if "temp" in dfv.columns:
    f = ((dfv["temp"] < -10) | (dfv["temp"] > 45)).sum()
    checks.append(("temp_range", f))
if "rhum" in dfv.columns:
    f = ((dfv["rhum"] < 0) | (dfv["rhum"] > 100)).sum()
    checks.append(("rhum_range", f))
if "wspd" in dfv.columns:
    f = ((dfv["wspd"] < 0) | (dfv["wspd"] > 120)).sum()
    checks.append(("wspd_range", f))

total = sum(x[1] for x in checks)
passed = total == 0
add_result("range_check", passed, f"fails={checks}")

# 4. DATATYPE CHECK
passed = True
dtype = []
if "trans_id" in dfv.columns:
    dtype.append(("trans_id_string", str(dfv["trans_id"].dtype) == "string"))
if "pay_amount" in dfv.columns:
    dtype.append(("pay_amount_numeric", np.issubdtype(dfv["pay_amount"].dtype, np.number)))
if "tap_in_time" in dfv.columns:
    dtype.append(("tap_in_datetime", np.issubdtype(dfv["tap_in_time"].dtype, np.datetime64)))
if "time" in dfv.columns:
    dtype.append(("weather_datetime", np.issubdtype(dfv["time"].dtype, np.datetime64)))

passed = all(x[1] for x in dtype)
add_result("datatype_check", passed, f"checks={dtype}")

# 5. REFERENTIAL INTEGRITY CORRIDOR
passed = True
corr = []
if "corridor_id" in dfv.columns and "corridor_name" in dfv.columns:
    nulls = dfv[dfv["corridor_id"].notna()]["corridor_name"].isna().sum()
    uniq = dfv.dropna(subset=["corridor_id","corridor_name"]).groupby("corridor_id")["corridor_name"].nunique()
    multi = (uniq > 1).sum()
    corr = [("null_corridor_name", nulls),("multi_corridor_name", multi)]
    passed = nulls == 0 and multi == 0
    add_result("referential_integrity_corridor", passed, f"fails={corr}")
else:
    add_result("referential_integrity_corridor", False, "corridor columns not found")

# 6. DISTRIBUTION SKEW
passed = True
skew = {}
main = ["pay_amount","feat_trip_minutes","feat_dist_km","temp","rhum"]
cols = [c for c in main if c in dfv.columns]
for c in cols:
    v = pd.to_numeric(dfv[c], errors="coerce")
    skew[c] = v.skew() if v.notna().sum() > 3 else None

fail_cols = [c for c, v in skew.items() if v is not None and abs(v) > 5]
passed = len(fail_cols) == 0
add_result("distribution_skew", passed, f"skew={skew}")

# OUTPUT REPORT
val_report = pd.DataFrame(results)
display(val_report)

,rule,passed,detail
0,uniqueness_trans_id,1,duplicates=0
1,null_check_critical,1,"null_total=0 cols=['trans_id', 'tap_in_time', ..."
2,range_check,1,"fails=[('pay_amount_range', np.int64(0)), ('te..."
3,datatype_check,1,"checks=[('trans_id_string', True), ('pay_amoun..."
4,referential_integrity_corridor,0,"fails=[('null_corridor_name', np.int64(0)), ('..."
5,distribution_skew,0,skew={'pay_amount': np.float64(0.8304173969177...


In [55]:
# CELL — BUAT FOLDER PROCESSED
import os
os.makedirs("/content/drive/MyDrive/bigdata_final_project/datalake/processed_ETL", exist_ok=True)
print("Folder siap.")

# CELL — EXPORT KE CSV
out_path = "/content/drive/MyDrive/bigdata_final_project/datalake/processed_ETL/transformed_etl.csv"
df.to_csv(out_path, index=False)
print("Rows:", df.shape[0])
print("Cols:", df.shape[1])
display(df.head(5))

Folder siap.
Rows: 189500
Cols: 48


,trans_id,pay_card_id,pay_card_bank,pay_card_name,pay_card_sex,pay_card_birth_date,corridor_id,corridor_name,direction,tap_in_stops,tap_in_stops_name,tap_in_stops_lat,tap_in_stops_lon,stop_start_seq,tap_in_time,tap_out_stops,tap_out_stops_name,tap_out_stops_lat,tap_out_stops_lon,stop_end_seq,tap_out_time,pay_amount,pay_amount_norm,tap_in_stops_lat_norm,pay_card_bank_enc,pay_card_sex_enc,corridor_name_enc,direction_enc,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,temp_norm,rhum_norm,coco_enc,feat_hour,feat_dow,feat_is_weekend,feat_trip_minutes,feat_dist_km,feat_fare_per_km,feat_is_rain,feat_temp_rhum_index
0,QRFN941O7B02JW,4433384770639489,dki,Tasnim Wastuti,M,1970-01-01 00:00:00.000002001,8,Jelambar - Juanda,1.0,0,Pecenongan,-6.167710,106.82819,1,2023-04-01 05:14:35,0,S Parman Podomoro City Arah Barat,-6.175587,106.79223,5,2023-04-01 07:52:46,3500.0,0.4,0.704476,2,1,51,1,2023-04-01 05:00:00,28.2,23.8,77.0,0.3,270.0,21.6,1009.4,8.0,0.503876,0.610619,6,5,5,1,158.183333,4.070741,859.794244,1,28.97
1,GDAB031P6R25GB,676172803949,flazz,"Darsirah Saputra, M.TI.",F,1970-01-01 00:00:00.000002012,8,Taman Kota - Budi Luhur,1.0,0,Ps. Kemiri 1,-6.164976,106.74928,6,2023-04-01 05:17:20,0,Simpang Permata Meruya 2,-6.195059,106.73972,26,2023-04-01 06:59:11,0.0,0.0,0.714797,4,0,186,1,2023-04-01 05:00:00,28.2,23.8,77.0,0.3,270.0,21.6,1009.4,8.0,0.503876,0.610619,6,5,5,1,101.850000,3.508057,0.000000,1,28.97
2,CQBV852W9W00EB,3551283641131718,dki,Limar Saptono,F,1970-01-01 00:00:00.000002009,10,Tanjung Priok - PGC 2,1.0,0,Cawang UKI,-6.250309,106.87360,2,2023-04-01 05:30:41,0,Pedati Prumpung,-6.220160,106.87439,6,2023-04-01 06:04:48,3500.0,0.4,0.392650,2,0,199,1,2023-04-01 06:00:00,29.6,23.5,70.0,0.2,270.0,9.4,1009.0,7.0,0.612403,0.486726,5,5,5,1,34.116667,3.353553,1043.669216,1,30.30
3,OHON419E4K33YS,4652692559663,online,Atma Saputra,F,1970-01-01 00:00:00.000002001,8,Pondok Labu - Blok M,1.0,0,Blok M Jalur 3,-6.243426,106.80188,0,2023-04-01 05:50:47,0,Dapsus 1,-6.301135,106.79484,17,2023-04-01 08:16:57,3500.0,0.4,0.418635,5,0,113,1,2023-04-01 06:00:00,29.6,23.5,70.0,0.2,270.0,9.4,1009.0,7.0,0.612403,0.486726,5,5,5,1,146.166667,6.463954,541.464242,1,30.30
4,IFLV900L2S37ZI,373037148645006,emoney,Digdaya Puspasari,M,1970-01-01 00:00:00.000001967,8,unknown,1.0,0,St. Sawah Besar 1,-6.159888,106.82767,23,2023-04-01 06:27:54,0,SMK Jakpus 1,-6.175714,106.81961,35,2023-04-01 06:50:11,0.0,0.0,0.734005,3,1,216,1,2023-04-01 06:00:00,29.6,23.5,70.0,0.2,270.0,9.4,1009.0,7.0,0.612403,0.486726,5,6,5,1,22.283333,1.972499,0.000000,1,30.30


# **LOAD**

**Import Library**

In [56]:
import os
import time
import json
import sqlite3
import pandas as pd
from datetime import datetime

**Path DB dan Input csv (hasil transform)**

In [57]:
ROOT = "/content/drive/MyDrive/bigdata_final_project"
DB_PATH = f"{ROOT}/warehouse/bigdata_ETL_warehouse.db"
TRANSFORM_CSV = f"{ROOT}/datalake/processed_ETL/transformed_etl.csv"

os.makedirs(f"{ROOT}/warehouse", exist_ok=True)
os.makedirs(f"{ROOT}/etl_pipeline", exist_ok=True)

print("DB_PATH:", DB_PATH)
print("CSV:", TRANSFORM_CSV)

DB_PATH: /content/drive/MyDrive/bigdata_final_project/warehouse/bigdata_ETL_warehouse.db
CSV: /content/drive/MyDrive/bigdata_final_project/datalake/processed_ETL/transformed_etl.csv


**Load csv ke Dataframe**

In [58]:
df = pd.read_csv(TRANSFORM_CSV)
print("Rows:", df.shape[0])
print("Cols:", df.shape[1])
display(df.head(3))

Rows: 189500
Cols: 48


,trans_id,pay_card_id,pay_card_bank,pay_card_name,pay_card_sex,pay_card_birth_date,corridor_id,corridor_name,direction,tap_in_stops,tap_in_stops_name,tap_in_stops_lat,tap_in_stops_lon,stop_start_seq,tap_in_time,tap_out_stops,tap_out_stops_name,tap_out_stops_lat,tap_out_stops_lon,stop_end_seq,tap_out_time,pay_amount,pay_amount_norm,tap_in_stops_lat_norm,pay_card_bank_enc,pay_card_sex_enc,corridor_name_enc,direction_enc,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,temp_norm,rhum_norm,coco_enc,feat_hour,feat_dow,feat_is_weekend,feat_trip_minutes,feat_dist_km,feat_fare_per_km,feat_is_rain,feat_temp_rhum_index
0,QRFN941O7B02JW,4433384770639489,dki,Tasnim Wastuti,M,1970-01-01 00:00:00.000002001,8,Jelambar - Juanda,1.0,0,Pecenongan,-6.167710,106.82819,1,2023-04-01 05:14:35,0,S Parman Podomoro City Arah Barat,-6.175587,106.79223,5,2023-04-01 07:52:46,3500.0,0.4,0.704476,2,1,51,1,2023-04-01 05:00:00,28.2,23.8,77.0,0.3,270.0,21.6,1009.4,8.0,0.503876,0.610619,6,5,5,1,158.183333,4.070741,859.794244,1,28.97
1,GDAB031P6R25GB,676172803949,flazz,"Darsirah Saputra, M.TI.",F,1970-01-01 00:00:00.000002012,8,Taman Kota - Budi Luhur,1.0,0,Ps. Kemiri 1,-6.164976,106.74928,6,2023-04-01 05:17:20,0,Simpang Permata Meruya 2,-6.195059,106.73972,26,2023-04-01 06:59:11,0.0,0.0,0.714797,4,0,186,1,2023-04-01 05:00:00,28.2,23.8,77.0,0.3,270.0,21.6,1009.4,8.0,0.503876,0.610619,6,5,5,1,101.850000,3.508057,0.000000,1,28.97
2,CQBV852W9W00EB,3551283641131718,dki,Limar Saptono,F,1970-01-01 00:00:00.000002009,10,Tanjung Priok - PGC 2,1.0,0,Cawang UKI,-6.250309,106.87360,2,2023-04-01 05:30:41,0,Pedati Prumpung,-6.220160,106.87439,6,2023-04-01 06:04:48,3500.0,0.4,0.392650,2,0,199,1,2023-04-01 06:00:00,29.6,23.5,70.0,0.2,270.0,9.4,1009.0,7.0,0.612403,0.486726,5,5,5,1,34.116667,3.353553,1043.669216,1,30.30


**Define Star Schema (Dim dan Fact)**

In [59]:
# dim_corridor: 1 baris per corridor
dim_corridor = (
    df[["corridor_id","corridor_name"]]
    .dropna(subset=["corridor_id"])
    .drop_duplicates(subset=["corridor_id"])
    .copy()
)

# dim_card: 1 baris per kartu (pay_card_id)
dim_card_cols = [c for c in ["pay_card_id","pay_card_bank","pay_card_sex","pay_card_name","pay_card_birth_date"] if c in df.columns]
dim_card = (
    df[dim_card_cols]
    .dropna(subset=["pay_card_id"])
    .drop_duplicates(subset=["pay_card_id"])
    .copy()
)

# fact_trip: transaksi
fact_cols = [
    "trans_id","pay_card_id","corridor_id",
    "tap_in_time","tap_out_time","pay_amount",
    "tap_in_stops","tap_out_stops",
    "feat_hour","feat_dow","feat_is_weekend","feat_trip_minutes",
    "feat_dist_km","feat_fare_per_km","feat_is_rain","feat_temp_rhum_index",
    "temp","rhum","prcp","wspd","pres","coco"
]
fact_cols = [c for c in fact_cols if c in df.columns]
fact_trip = df[fact_cols].drop_duplicates(subset=["trans_id"]).copy()

print("dim_corridor:", dim_corridor.shape)
print("dim_card:", dim_card.shape)
print("fact_trip:", fact_trip.shape)

dim_corridor: (14, 2)
dim_card: (10000, 5)
fact_trip: (189500, 22)


**Create Tables (PK + FK) di sqlite**

In [60]:
def run_sql(conn, sql):
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.executescript(sql)
    conn.commit()

schema_sql = """
DROP TABLE IF EXISTS fact_trip;
DROP TABLE IF EXISTS dim_card;
DROP TABLE IF EXISTS dim_corridor;

CREATE TABLE dim_corridor (
  corridor_id INTEGER PRIMARY KEY,
  corridor_name TEXT
);

CREATE TABLE dim_card (
  pay_card_id TEXT PRIMARY KEY,
  pay_card_bank TEXT,
  pay_card_sex TEXT,
  pay_card_name TEXT,
  pay_card_birth_date TEXT
);

CREATE TABLE fact_trip (
  trans_id TEXT PRIMARY KEY,
  pay_card_id TEXT,
  corridor_id INTEGER,
  tap_in_time TEXT,
  tap_out_time TEXT,
  pay_amount REAL,
  tap_in_stops INTEGER,
  tap_out_stops INTEGER,

  feat_hour INTEGER,
  feat_dow INTEGER,
  feat_is_weekend INTEGER,
  feat_trip_minutes REAL,
  feat_dist_km REAL,
  feat_fare_per_km REAL,
  feat_is_rain INTEGER,
  feat_temp_rhum_index REAL,

  temp REAL,
  rhum REAL,
  prcp REAL,
  wspd REAL,
  pres REAL,
  coco REAL,

  FOREIGN KEY (pay_card_id) REFERENCES dim_card(pay_card_id),
  FOREIGN KEY (corridor_id) REFERENCES dim_corridor(corridor_id)
);
"""

conn = sqlite3.connect(DB_PATH)
run_sql(conn, schema_sql)
print("Schema created.")

Schema created.


**Insert Data dan Log**

In [ ]:
LOG_PATH = f"{ROOT}/etl_pipeline/load_log.jsonl"

t0 = time.time()
start = datetime.now().isoformat()

# simpan ke sqlite
dim_corridor.to_sql("dim_corridor", conn, if_exists="append", index=False)
dim_card.to_sql("dim_card", conn, if_exists="append", index=False)
fact_trip.to_sql("fact_trip", conn, if_exists="append", index=False)

exec_s = round(time.time() - t0, 2)

log = {
    "stage": "load",
    "status": "success",
    "start": start,
    "seconds": exec_s,
    "db_path": DB_PATH,
    "rows_dim_corridor": int(dim_corridor.shape[0]),
    "rows_dim_card": int(dim_card.shape[0]),
    "rows_fact_trip": int(fact_trip.shape[0]),
}
with open(LOG_PATH, "a") as f:
    f.write(json.dumps(log) + "\n")

print("LOAD OK")
print("Seconds:", exec_s)
print("Log:", LOG_PATH)

**Query SQL Analitik**

In [ ]:
queries = {
  "Q1_total_trips": """
    SELECT COUNT(*) AS total_trips FROM fact_trip;
  """,
  "Q2_total_revenue": """
    SELECT ROUND(SUM(pay_amount),2) AS total_revenue FROM fact_trip;
  """,
  "Q3_top5_corridor_by_trips": """
    SELECT d.corridor_name, COUNT(*) AS trips
    FROM fact_trip f
    JOIN dim_corridor d ON f.corridor_id = d.corridor_id
    GROUP BY d.corridor_name
    ORDER BY trips DESC
    LIMIT 5;
  """,
  "Q4_avg_fare_per_trip_by_corridor_top5": """
    SELECT d.corridor_name, ROUND(AVG(f.pay_amount),2) AS avg_fare
    FROM fact_trip f
    JOIN dim_corridor d ON f.corridor_id = d.corridor_id
    GROUP BY d.corridor_name
    ORDER BY avg_fare DESC
    LIMIT 5;
  """,
  "Q5_trips_by_hour": """
    SELECT feat_hour AS hour, COUNT(*) AS trips
    FROM fact_trip
    GROUP BY feat_hour
    ORDER BY hour;
  """,
  "Q6_weekend_vs_weekday_trips": """
    SELECT feat_is_weekend AS is_weekend, COUNT(*) AS trips
    FROM fact_trip
    GROUP BY feat_is_weekend
    ORDER BY is_weekend;
  """,
  "Q7_trips_when_rain": """
    SELECT feat_is_rain AS is_rain, COUNT(*) AS trips
    FROM fact_trip
    GROUP BY feat_is_rain
    ORDER BY is_rain;
  """,
  "Q8_avg_temp_and_revenue_by_weather_code": """
    SELECT coco,
           ROUND(AVG(temp),2) AS avg_temp,
           ROUND(SUM(pay_amount),2) AS revenue,
           COUNT(*) AS trips
    FROM fact_trip
    GROUP BY coco
    ORDER BY trips DESC
    LIMIT 10;
  """
}

for k, q in queries.items():
    print(k)
    display(pd.read_sql_query(q, conn))

Q1_total_trips


,total_trips
0,189500


Q2_total_revenue


,total_revenue
0,415273250.0


Q3_top5_corridor_by_trips


...

,is_weekend,trips
0,0,171000
1,1,18500


Q7_trips_when_rain


,is_rain,trips
0,0,135986
1,1,53514


Q8_avg_temp_and_revenue_by_weather_code


,coco,avg_temp,revenue,trips
0,3.0,27.34,283445750.0,129323
1,17.0,30.63,32019750.0,14614
2,8.0,28.98,29568000.0,13514
3,7.0,30.54,27616750.0,12573
4,9.0,29.14,23964500.0,10932
5,2.0,25.68,9931250.0,4618
6,1.0,27.95,4534250.0,2045
7,18.0,30.67,4193000.0,1881


**Close Connection**

In [ ]:
conn.close()
print("DB saved:", DB_PATH)

DB saved: /content/drive/MyDrive/bigdata_final_project/warehouse/bigdata_ETL_warehouse.db
